In [ ]:
import open3d as o3d
import numpy as np
import copy
import os
import sys

# monkey patches visualization and provides helpers to load geometries
sys.path.append('..')
import open3d_tutorial as o3dtut
# change to True if you want to interact with the visualization windows
o3dtut.interactive = not "CI" in os.environ

# Generalized ICP Registration
This tutorial demonstrates an ICP variant that combine the  [Point-to-plane ICP](../pipelines/icp_registration.ipynb#Point-to-plane-ICP) and [Point-to-point ICP](../pipelines/icp_registration.ipynb#Point-to-point-ICP)  algorithms into a single probabilistic framework. It implements the algorithm of [\[Segal2009\]](../reference.html#segal2009). 

The framework was originally used to model locally planar surface structure from both scans instead of just the model scan as is typically done with the point-to-plane method. This can be thought of as plane-to-plane, but, as its name suggests it can be extended to account for other types of approximations. On the original publication it's shown that both  [Point-to-plane ICP](../pipelines/icp_registration.ipynb#Point-to-plane-ICP) and [Point-to-point ICP](../pipelines/icp_registration.ipynb#Point-to-point-ICP) are special cases of the Generalized-icp framework.

<div class="alert alert-info">
 
**Note:** 

This tutorial and the implementation of the Generalized ICP algorithm in `Open3D` was contributed by **Ignacio Vizzo** and **Cyrill Stachniss** from the University of Bonn.

</div>

## Helper visualization function
The function below visualizes a target point cloud and a source point cloud transformed with an alignment transformation. The target point cloud and the source point cloud are painted with red and green colors respectively.
Additionally, two line sets are added to the visualization to show the "GT" surface where the points were sampled

In [ ]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.transform(transformation)
    # Add lines
    split = len(source.points) // 2
    lines = [[0, split], [2 * split, 0]]
    source_line = o3d.geometry.LineSet(
        points=source_temp.points,
        lines=o3d.utility.Vector2iVector(lines),
    )
    target_line = o3d.geometry.LineSet(
        points=target_temp.points,
        lines=o3d.utility.Vector2iVector(lines),
    )
    # Paint geometries
    source_temp.paint_uniform_color([1, 0, 0])
    source_line.paint_uniform_color([1, 0, 0])
    target_line.paint_uniform_color([0, 1, 0])
    target_temp.paint_uniform_color([0, 1, 0])
    o3d.visualization.draw_geometries(
        [source_temp, source_line, target_temp, target_line],
        point_show_normal=True,
        front=[0.0, 0.0, 1.0],
        lookat=[1.13, -0.06, 0.0],
        up=[0.0, 1.0, 0.0],
        zoom=1.14)

## Input

To best show the application of `Generalized-icp` we will use the same example shown in `Fig. 1.` on the original paper of [\[Segal2009\]](../reference.html#segal2009).

For simplification we stick to the `2D` example

In [ ]:
print("1. Create the same pointclouds shown in the original paper")
points = [
    [0.0, 0.0, 0.0],
    [1.0, 0.0, 0.0],
    [2.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 2.0, 0.0],
]

normals = [
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 1.0, 0.0],
    [1.0, 0.0, 0.0],
    [1.0, 0.0, 0.0],
]
source = o3d.geometry.PointCloud()
source.points = o3d.utility.Vector3dVector(points)
source.normals = o3d.utility.Vector3dVector(normals)
target = copy.deepcopy(source)
target.translate([+1.0, -2.0, 0.0])

# draw initial alignment
current_transformation = np.identity(4)
draw_registration_result(source, target, current_transformation)

## Distance Threshold

To ilustrate the greater convergence rate of the Generalized-icp algorithm we choose a high distance threshold.  When this threshold is big enough, then the "corner" point will get the wrong correspondence when using traditional ICP methods and will produce a wrong result.

Be aware that if you pick a small distance threshold (like `2.0`) then, the "corner" point of the example won't be associated initially to any point and thus you will converge to the right solution.

In [ ]:
distance_th = 3.0

## Point-to-point ICP
We first run [Point-to-point ICP](../pipelines/icp_registration.ipynb#Point-to-point-ICP) as a baseline approach. Due the "corner" point we end up with a bad aligment result.

In [ ]:
# point to point ICP
current_transformation = np.identity(4)
print("2. Point-to-point ICP registration is applied on original point")
print("   clouds to refine the alignment.")
p2p_result_icp = o3d.pipelines.registration.registration_icp(
    source, target, distance_th, current_transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(p2p_result_icp)
draw_registration_result(source, target, p2p_result_icp.transformation)

## Point-to-plane ICP
We first run [Point-to-plane ICP](../pipelines/icp_registration.ipynb#Point-to-plane-ICP) as a baseline approach. Due the "corner" point we end up with a bad aligment result.

In [ ]:
# point to plane ICP
current_transformation = np.identity(4)
print("3. Point-to-plane ICP registration is applied on original point")
print("   clouds to refine the alignment.")
p2l_result_icp = o3d.pipelines.registration.registration_icp(
    source, target, distance_th, current_transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPlane())
print(p2l_result_icp)
draw_registration_result(source, target, p2l_result_icp.transformation)

## Plane-to-plane ICP (Generalized ICP)

We now show how the `Plane-to-plane` registration scheme converge to the right solution under the same assumptions and the same registration parameters

In [ ]:
# Generelized ICP
current_transformation = np.identity(4)
print("4. Generalized ICP registration is applied on original point")
print("   clouds to refine the alignment.")
result_icp = o3d.pipelines.registration.registration_generalized_icp(
    source, target, distance_th, current_transformation)
print(result_icp)
draw_registration_result(source, target, result_icp.transformation)

## Point Covariance Estimation

You can go as wild as you wish. The `registration_generalized_icp` implementation will compute the per-point covariance matrices following the original paper descriptions(and thus, running a Plane-to-plane scheme) but you are not restricted to it. You can compute the covariances using your favorite algorithm and then call the `registration_generalized_icp` function

In this case we will show that the `Point-to-point ICP` is an specific case of the Generalized-icp following equation (3) of the original paper

In [ ]:
# You can use any 3x3 matrix here, or compute it from the input data
source_covariances = [
    np.zeros((3, 3)),
    np.zeros((3, 3)),
    np.zeros((3, 3)),
    np.zeros((3, 3)),
    np.zeros((3, 3)),
]
target_covariances = [
    np.eye(3),
    np.eye(3),
    np.eye(3),
    np.eye(3),
    np.eye(3),
]
source_c = copy.deepcopy(source)
source_c.covariances = o3d.utility.Matrix3dVector(source_covariances)

target_c = copy.deepcopy(target)
target_c.covariances = o3d.utility.Matrix3dVector(target_covariances)
print("Source has covariances:", source_c.has_covariances())
print("Target has covariances:", target_c.has_covariances())

In [ ]:
current_transformation = np.identity(4)
print(
    "5. Generalized ICP, Point-to-point registration is applied on original point"
)
print("   clouds to refine the alignment.")
gicp_p2p_result_icp = o3d.pipelines.registration.registration_generalized_icp(
    source_c, target_c, distance_th, current_transformation)
draw_registration_result(source, target, gicp_p2p_result_icp.transformation)

print("Original Point-to-point results:\n", p2p_result_icp)
print("Generalized ICP Point-to-point results:\n", gicp_p2p_result_icp)